In [1]:
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
import os

# Загрузка перементов окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("load_olapcube.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

def calculate_aggregate_count():
    try:
        # Суммирование mi17_count и mi8t_count
        query_fleet = """
        SELECT 
            SUM(mi17_count) AS total_mi17_count, 
            SUM(mi8t_count) AS total_mi8t_count 
        FROM fleet
        """
        result_fleet = client.execute(query_fleet)
        total_mi17_count, total_mi8t_count = result_fleet[0]
        logger.info(f"Сумма mi17_count: {total_mi17_count}, сумма mi8t_count: {total_mi8t_count}")

        # Получение значений из Agregat
        query_agregat = """
        SELECT 
            items_per_ac, 
            NR_Mi17 
        FROM Agregat 
        LIMIT 1
        """
        result_agregat = client.execute(query_agregat)
        if not result_agregat:
            logger.error("Таблица Agregat пуста. Невозможно вычислить aggregate_count.")
            return None
        items_per_ac, NR_Mi17 = result_agregat[0]
        logger.info(f"items_per_ac: {items_per_ac}, NR_Mi17: {NR_Mi17}")

        if NR_Mi17 == 0:
            logger.error("NR_Mi17 равно 0. Деление невозможно.")
            return None

        # Получение monthly_flight_hours
        query_flight_hours = """
        SELECT 
            monthly_flight_hours 
        FROM flight_hours 
        LIMIT 1
        """
        result_flight_hours = client.execute(query_flight_hours)
        if not result_flight_hours:
            logger.error("Таблица flight_hours пуста. Невозможно получить monthly_flight_hours.")
            return None
        monthly_flight_hours = result_flight_hours[0][0]
        logger.info(f"monthly_flight_hours: {monthly_flight_hours}")

        # Расчет aggregate_count
        aggregate_count = (total_mi17_count + total_mi8t_count) * 11 * 1.5 * (items_per_ac * monthly_flight_hours) / NR_Mi17
        aggregate_count = round(aggregate_count)
        logger.info(f"Вычисленный aggregate_count: {aggregate_count}")
        return aggregate_count

    except Exception as e:
        logger.error(f"Ошибка при вычислении aggregate_count: {e}", exc_info=True)
        return None

def get_agregat_values():
    try:
        query = "SELECT MRR_Mi8T, MRR_Mi17, MRR_2_Mi8T, MRR_2_Mi17, NR_Mi8T, NR_Mi17, BR_Mi8T, BR_Mi17 FROM Agregat LIMIT 1"
        result = client.execute(query)
        if not result:
            logger.error("Таблица Agregat пуста. Невозможно получить значения.")
            return None
        MRR_Mi8T, MRR_Mi17, MRR_2_Mi8T, MRR_2_Mi17, NR_Mi8T, NR_Mi17, BR_Mi8T, BR_Mi17 = result[0]
        logger.info("Получены значения из таблицы Agregat.")
        return {
            'MRR_Mi8T': MRR_Mi8T,
            'MRR_Mi17': MRR_Mi17,
            'MRR_2_Mi8T': MRR_2_Mi8T,
            'MRR_2_Mi17': MRR_2_Mi17,
            'NR_Mi8T': NR_Mi8T,
            'NR_Mi17': NR_Mi17,
            'BR_Mi8T': BR_Mi8T,
            'BR_Mi17': BR_Mi17
        }
    except Exception as e:
        logger.error(f"Ошибка при получении значений из Agregat: {e}", exc_info=True)
        return None

def get_daily_flight_hours():
    try:
        query = "SELECT daily_flight_hours FROM flight_hours LIMIT 1"
        result = client.execute(query)
        if not result:
            logger.error("Таблица flight_hours пуста. Невозможно получить daily_flight_hours.")
            return None
        daily_flight_hours = result[0][0]
        logger.info(f"Получено daily_flight_hours: {daily_flight_hours}")
        return daily_flight_hours
    except Exception as e:
        logger.error(f"Ошибка при получении daily_flight_hours: {e}", exc_info=True)
        return None

def insert_into_olapcube_vnv_with_dates(aggregate_count, agregat_values, daily_flight_hours):
    try:
        if aggregate_count is None or agregat_values is None or daily_flight_hours is None:
            logger.error("Не все данные для вставки в OlapCube_VNV доступны.")
            return

        # Подготовка данных
        threshold = agregat_values['MRR_Mi8T'] if agregat_values['MRR_Mi8T'] and agregat_values['MRR_Mi8T'] != 0 else agregat_values['MRR_Mi17']
        oh = agregat_values['MRR_2_Mi8T'] if agregat_values['MRR_2_Mi8T'] and agregat_values['MRR_2_Mi8T'] != 0 else agregat_values['MRR_2_Mi17']
        ll = agregat_values['NR_Mi8T'] if agregat_values['NR_Mi8T'] and agregat_values['NR_Mi8T'] != 0 else agregat_values['NR_Mi17']
        BR = agregat_values['BR_Mi8T'] if agregat_values['BR_Mi8T'] and agregat_values['BR_Mi8T'] != 0 else agregat_values['BR_Mi17']

        # Получение всех уникальных дат
        query_dates = f"SELECT DISTINCT Dates FROM {database_name}.OlapCube_VNV ORDER BY Dates ASC"
        dates = client.execute(query_dates)
        if not dates:
            logger.error("Поле Dates в таблице OlapCube_VNV пусто.")
            return
        dates = [date[0] for date in dates]
        logger.info(f"Получено {len(dates)} уникальных дат.")

        # Генерация серийных номеров
        serial_numbers = [f"S{i + 1}" for i in range(aggregate_count)]
        logger.info(f"Сгенерировано {len(serial_numbers)} серийных номеров.")

        # Подготовка данных для вставки
        insert_query = f"""
        INSERT INTO {database_name}.OlapCube_VNV (
            serialno, Dates, threshold, oh, ll, BR, daily_flight_hours, Status_P, sne, ppr
        ) VALUES
        """
        data_to_insert = []
        for serial in serial_numbers:
            for idx, date in enumerate(dates):
                if idx == 0:
                    record = [
                        serial, date, threshold, oh, ll, BR, daily_flight_hours, "Неактивно", 0, 0
                    ]
                else:
                    record = [
                        serial, date, threshold, oh, ll, BR, daily_flight_hours, None, None, None
                    ]
                data_to_insert.append(record)

        # Вставка данных
        client.execute(insert_query, data_to_insert, types_check=True)
        logger.info(f"Успешно вставлено {len(data_to_insert)} записей в OlapCube_VNV.")

    except Exception as e:
        logger.error(f"Ошибка при вставке данных в OlapCube_VNV: {e}", exc_info=True)

def main():
    logger.info("Начинаю вычисление aggregate_count.")
    aggregate_count = calculate_aggregate_count()

    if aggregate_count is not None:
        logger.info(f"aggregate_count рассчитан: {aggregate_count}")

        logger.info("Получаю значения из таблицы Agregat.")
        agregat_values = get_agregat_values()

        logger.info("Получаю daily_flight_hours из flight_hours.")
        daily_flight_hours = get_daily_flight_hours()

        logger.info("Начинаю вставку данных в OlapCube_VNV.")
        insert_into_olapcube_vnv_with_dates(aggregate_count, agregat_values, daily_flight_hours)
    else:
        logger.error("Не удалось рассчитать aggregate_count. Пропускаем вставку в OlapCube_VNV.")

if __name__ == "__main__":
    main()


2025-07-08 17:03:11,995 - INFO - Начинаю вычисление aggregate_count.
2025-07-08 17:03:12,004 - INFO - Сумма mi17_count: 777.0, сумма mi8t_count: 1006.0
2025-07-08 17:03:12,007 - INFO - items_per_ac: 1.0, NR_Mi17: 6000.0
2025-07-08 17:03:12,011 - INFO - monthly_flight_hours: 48.0
2025-07-08 17:03:12,011 - INFO - Вычисленный aggregate_count: 235
2025-07-08 17:03:12,011 - INFO - aggregate_count рассчитан: 235
2025-07-08 17:03:12,012 - INFO - Получаю значения из таблицы Agregat.
2025-07-08 17:03:12,016 - INFO - Получены значения из таблицы Agregat.
2025-07-08 17:03:12,016 - INFO - Получаю daily_flight_hours из flight_hours.
2025-07-08 17:03:12,020 - INFO - Получено daily_flight_hours: 1.58
2025-07-08 17:03:12,020 - INFO - Начинаю вставку данных в OlapCube_VNV.
2025-07-08 17:03:12,028 - INFO - Получено 4001 уникальных дат.
2025-07-08 17:03:12,029 - INFO - Сгенерировано 235 серийных номеров.
2025-07-08 17:03:19,563 - INFO - Успешно вставлено 940235 записей в OlapCube_VNV.


In [2]:
import os
from clickhouse_driver import Client
import logging

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
CLICKHOUSE_HOST = '10.95.19.132'
CLICKHOUSE_PORT = 9000
CLICKHOUSE_USER = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
DATABASE_NAME = 'default'
AGREGAT_TABLE_NAME = 'Agregat'
OLAP_TABLE_NAME = 'OlapCube_VNV'
REPAIR_TIME_COLUMN = 'RepairTime'
REPAIR_TIME_TYPE = 'Nullable(Float64)'  # Измените тип при необходимости

def connect_to_clickhouse():
    """
    Устанавливает соединение с ClickHouse и возвращает объект клиента.
    """
    try:
        client = Client(
            host=CLICKHOUSE_HOST,
            port=CLICKHOUSE_PORT,
            user=CLICKHOUSE_USER,
            password=CLICKHOUSE_PASSWORD,
            database=DATABASE_NAME
        )
        logging.info("Подключение к ClickHouse успешно установлено")
        return client
    except Exception as e:
        logging.error(f"Не удалось подключиться к ClickHouse: {e}")
        raise

def check_and_add_column(client, table_name, column_name, column_type):
    """
    Проверяет наличие столбца в таблице и добавляет его, если отсутствует.
    """
    try:
        query = f"""
            SELECT COUNT(*)
            FROM system.columns
            WHERE database = '{DATABASE_NAME}'
              AND table = '{table_name}'
              AND name = '{column_name}'
        """
        result = client.execute(query)
        column_exists = result[0][0] > 0
        if column_exists:
            logging.info(f"Столбец '{column_name}' уже существует в таблице '{table_name}'")
        else:
            add_column_query = f"""
                ALTER TABLE {DATABASE_NAME}.{table_name}
                ADD COLUMN {column_name} {column_type}
            """
            client.execute(add_column_query)
            logging.info(f"Столбец '{column_name}' успешно добавлен в таблицу '{table_name}'")
    except Exception as e:
        logging.error(f"Ошибка при проверке или добавлении столбца '{column_name}' в таблицу '{table_name}': {e}")
        raise

def get_repair_time(client, agregat_table):
    """
    Получает значение RepairTime из таблицы Agregat.
    """
    try:
        query = f"""
            SELECT RepairTime
            FROM {DATABASE_NAME}.{agregat_table}
            LIMIT 1
        """
        result = client.execute(query)
        if not result:
            raise ValueError(f"Таблица '{agregat_table}' пуста или столбец '{REPAIR_TIME_COLUMN}' отсутствует.")
        repair_time = result[0][0]
        logging.info(f"Получено значение {REPAIR_TIME_COLUMN}: {repair_time}")
        return repair_time
    except Exception as e:
        logging.error(f"Ошибка при получении {REPAIR_TIME_COLUMN} из таблицы '{agregat_table}': {e}")
        raise

def update_repair_time(client, table_name, repair_time):
    """
    Обновляет столбец RepairTime в таблице OlapCube_VNV для записей, где RepairTime равен NULL.
    """
    try:
        # Используем выражение, которое явно читает столбец RepairTime
        update_query = f"""
            ALTER TABLE {DATABASE_NAME}.{table_name}
            UPDATE {REPAIR_TIME_COLUMN} = {repair_time} + (IFNULL({REPAIR_TIME_COLUMN}, 0) * 0)
            WHERE {REPAIR_TIME_COLUMN} IS NULL
        """
        client.execute(update_query)
        logging.info(f"Столбец '{REPAIR_TIME_COLUMN}' успешно обновлён в таблице '{table_name}'")
    except Exception as e:
        logging.error(f"Ошибка при обновлении столбца '{REPAIR_TIME_COLUMN}' в таблице '{table_name}': {e}")
        raise

def get_table_schema(client, table_name):
    """
    Получает и возвращает схему таблицы.
    """
    try:
        query = f"SHOW CREATE TABLE {DATABASE_NAME}.{table_name}"
        schema = client.execute(query)[0][0]
        logging.info(f"Схема таблицы '{table_name}':\n{schema}")
        return schema
    except Exception as e:
        logging.error(f"Ошибка при получении схемы таблицы '{table_name}': {e}")
        raise

def main():
    # Подключение к ClickHouse
    client = connect_to_clickhouse()
    
    # Проверка схемы таблицы OlapCube_VNV
    try:
        get_table_schema(client, OLAP_TABLE_NAME)
    except Exception as e:
        logging.error(f"Произошла ошибка при проверке схемы таблицы '{OLAP_TABLE_NAME}': {e}", exc_info=True)
        return
    
    # Проверка наличия столбца RepairTime и его добавление при необходимости
    try:
        check_and_add_column(client, OLAP_TABLE_NAME, REPAIR_TIME_COLUMN, REPAIR_TIME_TYPE)
    except Exception as e:
        logging.error(f"Произошла ошибка при проверке или добавлении столбца '{REPAIR_TIME_COLUMN}': {e}", exc_info=True)
        return
    
    # Получение значения RepairTime из таблицы Agregat
    try:
        repair_time = get_repair_time(client, AGREGAT_TABLE_NAME)
    except Exception as e:
        logging.error(f"Произошла ошибка при получении значения '{REPAIR_TIME_COLUMN}': {e}", exc_info=True)
        return
    
    # Обновление столбца RepairTime в таблице OlapCube_VNV
    try:
        update_repair_time(client, OLAP_TABLE_NAME, repair_time)
    except Exception as e:
        logging.error(f"Произошла ошибка при обновлении столбца '{REPAIR_TIME_COLUMN}': {e}", exc_info=True)

if __name__ == "__main__":
    main()


2025-07-08 17:03:19,589 - INFO - Подключение к ClickHouse успешно установлено
2025-07-08 17:03:19,594 - INFO - Схема таблицы 'OlapCube_VNV':
CREATE TABLE default.OlapCube_VNV
(
    `serialno` String,
    `Dates` Date,
    `Status_P` Nullable(String),
    `repair_days` Nullable(Float64),
    `sne` Nullable(Float64),
    `ppr` Nullable(Float64),
    `Status` Nullable(String),
    `daily_flight_hours` Nullable(Float64),
    `daily_flight_hours_f` Nullable(Float64),
    `BR` Nullable(Float64),
    `ll` Nullable(Float64),
    `oh` Nullable(Float64),
    `threshold` Nullable(Float64),
    `Effectivity` Nullable(String),
    `mi8t_count` Nullable(Float64),
    `mi17_count` Nullable(Float64),
    `balance_mi8t` Float64,
    `balance_mi17` Float64,
    `balance_total` Float64,
    `stock_mi8t` Float64,
    `stock_mi17` Float64,
    `stock_total` Float64,
    `balance_empty` Float64,
    `stock_empty` Float64,
    `location` String,
    `ac_typ` String
)
ENGINE = ReplacingMergeTree
ORDER BY (ser

In [3]:
import os
from clickhouse_driver import Client
import logging

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = 'default'
fleet_table_name = 'fleet'
olap_table_name = 'OlapCube_VNV'

# Функция для преобразования названий месяцев на русском в номера
def get_month_number(month_name):
    months = {
        "Январь": 1,
        "Февраль": 2,
        "Март": 3,
        "Апрель": 4,
        "Май": 5,
        "Июнь": 6,
        "Июль": 7,
        "Август": 8,
        "Сентябрь": 9,
        "Октябрь": 10,
        "Ноябрь": 11,
        "Декабрь": 12
    }
    return months.get(month_name, None)

# Функция для получения данных из таблицы fleet с приведением типов
def get_fleet_data(client):
    try:
        fleet_query = f"SELECT month, mi8t_count, mi17_count FROM {database_name}.{fleet_table_name}"
        fleet_results = client.execute(fleet_query)
        fleet_dict = {}
        for row in fleet_results:
            month_name = row[0]
            mi8t_count = row[1]
            mi17_count = row[2]
            month_number = get_month_number(month_name)
            if month_number:
                # Приведение mi8t_count и mi17_count к типу Float64, обработка None
                try:
                    mi8t_count = float(mi8t_count) if mi8t_count is not None else None
                except ValueError:
                    logging.warning(f"mi8t_count не является числом для месяца: {month_name}, значение: {mi8t_count}")
                    mi8t_count = None
                try:
                    mi17_count = float(mi17_count) if mi17_count is not None else None
                except ValueError:
                    logging.warning(f"mi17_count не является числом для месяца: {month_name}, значение: {mi17_count}")
                    mi17_count = None
                fleet_dict[month_number] = {
                    'mi8t_count': mi8t_count,
                    'mi17_count': mi17_count
                }
            else:
                logging.warning(f"Не удалось определить номер месяца для названия: {month_name}")
        logging.info(f"Получено данных из fleet: {len(fleet_dict)} месяцев")
        return fleet_dict
    except Exception as e:
        logging.error(f"Ошибка при получении данных из fleet: {e}")
        raise

# Функция для обновления полей mi8t_count и mi17_count в OlapCube_VNV на основании Dates
def update_olap_cube(client, fleet_dict):
    try:
        # Построение CASE выражений для mi8t_count и mi17_count
        mi8t_case_expr = "CASE "
        mi17_case_expr = "CASE "

        for month_number in range(1, 13):
            fleet_counts = fleet_dict.get(month_number, {'mi8t_count': 'NULL', 'mi17_count': 'NULL'})
            mi8t_value = fleet_counts['mi8t_count'] if fleet_counts['mi8t_count'] is not None else 'NULL'
            mi17_value = fleet_counts['mi17_count'] if fleet_counts['mi17_count'] is not None else 'NULL'

            mi8t_case_expr += f"WHEN toMonth(Dates) = {month_number} THEN {mi8t_value} "
            mi17_case_expr += f"WHEN toMonth(Dates) = {month_number} THEN {mi17_value} "

        mi8t_case_expr += "ELSE NULL END"
        mi17_case_expr += "ELSE NULL END"

        # Формирование запроса для обновления
        update_query = f"""
        ALTER TABLE {database_name}.{olap_table_name}
        UPDATE
            mi8t_count = {mi8t_case_expr},
            mi17_count = {mi17_case_expr}
        WHERE 1
        """

        logging.info("Начинается обновление полей mi8t_count и mi17_count в OlapCube_VNV")
        client.execute(update_query)
        logging.info("Поля mi8t_count и mi17_count успешно обновлены в OlapCube_VNV")
    except Exception as e:
        logging.error(f"Ошибка при обновлении OlapCube_VNV: {e}", exc_info=True)
        raise

# Подключение к ClickHouse с указанием базы данных
try:
    client = Client(
        host=clickhouse_host,
        port=clickhouse_port,
        user=clickhouse_user,
        password=clickhouse_password,
        database=database_name  # Добавлено указание базы данных
    )
    logging.info("Подключение к ClickHouse успешно установлено")
except Exception as e:
    logging.error(f"Не удалось подключиться к ClickHouse: {e}")
    raise

# Выполнение функций для получения данных и обновления таблицы
try:
    fleet_dict = get_fleet_data(client)
    update_olap_cube(client, fleet_dict)
    logging.info("Данные mi8t_count и mi17_count успешно обновлены на основании Dates в OlapCube_VNV")
except Exception as e:
    logging.error(f"Произошла ошибка при обновлении данных: {e}", exc_info=True)


2025-07-08 17:03:19,621 - INFO - Подключение к ClickHouse успешно установлено
2025-07-08 17:03:19,632 - INFO - Получено данных из fleet: 12 месяцев
2025-07-08 17:03:19,632 - INFO - Начинается обновление полей mi8t_count и mi17_count в OlapCube_VNV
2025-07-08 17:03:19,648 - INFO - Поля mi8t_count и mi17_count успешно обновлены в OlapCube_VNV
2025-07-08 17:03:19,648 - INFO - Данные mi8t_count и mi17_count успешно обновлены на основании Dates в OlapCube_VNV


In [4]:
import os
from clickhouse_driver import Client
import logging
from datetime import timedelta

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = 'default'
flight_hours_table_name = 'flight_hours'
olap_table_name = 'OlapCube_VNV'

# Функция для заполнения поля daily_flight_hours в таблице OLAP
def update_daily_flight_hours(client):
    try:
        # Получение значения daily_flight_hours из таблицы flight_hours
        flight_hours_query = f"""
            SELECT daily_flight_hours
            FROM {database_name}.{flight_hours_table_name}
            LIMIT 1
        """
        daily_flight_hours = client.execute(flight_hours_query)[0][0]
        logging.info(f"Получено значение daily_flight_hours: {daily_flight_hours}")
    except Exception as e:
        logging.error(f"Ошибка при получении daily_flight_hours из flight_hours: {e}")
        raise

    # Обновление поля daily_flight_hours в OLAP кубе для всех записей на всем протяжении дат
    try:
        update_query = f"""
            ALTER TABLE {database_name}.{olap_table_name}
            UPDATE daily_flight_hours = {daily_flight_hours}
            WHERE daily_flight_hours IS NULL
        """
        client.execute(update_query)
        logging.info(f"Поле daily_flight_hours успешно обновлено в {olap_table_name}")
    except Exception as e:
        logging.error(f"Ошибка при обновлении поля daily_flight_hours в {olap_table_name}: {e}")
        raise

# Подключение к ClickHouse с указанием базы данных
try:
    client = Client(
        host=clickhouse_host,
        port=clickhouse_port,
        user=clickhouse_user,
        password=clickhouse_password,
        database=database_name
    )
    logging.info("Подключение к ClickHouse успешно установлено")
except Exception as e:
    logging.error(f"Не удалось подключиться к ClickHouse: {e}")
    raise

# Обновление поля daily_flight_hours в OLAP кубе
try:
    update_daily_flight_hours(client)
    logging.info("Поле daily_flight_hours успешно обновлено в OLAP кубе")
except Exception as e:
    logging.error(f"Произошла ошибка при обновлении daily_flight_hours: {e}", exc_info=True)

2025-07-08 17:03:19,654 - INFO - Подключение к ClickHouse успешно установлено
2025-07-08 17:03:19,660 - INFO - Получено значение daily_flight_hours: 1.58
2025-07-08 17:03:19,671 - INFO - Поле daily_flight_hours успешно обновлено в OlapCube_VNV
2025-07-08 17:03:19,672 - INFO - Поле daily_flight_hours успешно обновлено в OLAP кубе


In [5]:
import os
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,  # Можно установить DEBUG для подробного логирования
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()

# Глобальные переменные
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Подключение к ClickHouse
client = Client(
    host=os.getenv('CLICKHOUSE_HOST', '10.95.19.132'),
    user=os.getenv('CLICKHOUSE_USER', 'default'),
    password=os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi'),
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)


def add_trigger_type_column(client, database_name):
    """
    Добавляет поле trigger_type в таблицу OlapCube_VNV, если оно еще не существует.
    """
    try:
        add_column_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV 
        ADD COLUMN IF NOT EXISTS trigger_type UInt8
        """
        client.execute(add_column_query)
        logger.info("Поле trigger_type успешно добавлено в таблицу OlapCube_VNV.")
    except Exception as e:
        logger.error(f"Ошибка при добавлении поля trigger_type: {e}", exc_info=True)
        raise


def get_agregat_data(client, database_name):
    """
    Извлекает данные из таблицы Agregat.
    """
    try:
        query_agregat = f"""
        SELECT 
            ComponentNumber, 
            NR_Mi8T, NR_Mi17, 
            MRR_Mi8T, MRR_Mi17, 
            MRR_2_Mi8T, MRR_2_Mi17
        FROM {database_name}.Agregat
        """
        agregat_data = client.execute(query_agregat)
        logger.info(f"Извлечено {len(agregat_data)} записей из таблицы Agregat.")
        return agregat_data
    except Exception as e:
        logger.error(f"Ошибка при извлечении данных из таблицы Agregat: {e}", exc_info=True)
        raise


def determine_trigger_type(agregat_data):
    """
    Определяет значения trigger_type на основе данных из Agregat.
    Возвращает два списка serialno для trigger_type=1 и trigger_type=0.
    """
    trigger_type_1_serialnos = []
    trigger_type_0_serialnos = []

    for row in agregat_data:
        component_number = row[0]
        NR_Mi8T = row[1]
        NR_Mi17 = row[2]
        MRR_Mi8T = row[3]
        MRR_Mi17 = row[4]
        MRR_2_Mi8T = row[5]
        MRR_2_Mi17 = row[6]

        # Определяем значение trigger_type
        trigger_type = 1 if (NR_Mi8T != NR_Mi17 or MRR_Mi8T != MRR_Mi17 or MRR_2_Mi8T != MRR_2_Mi17) else 0

        if trigger_type == 1:
            trigger_type_1_serialnos.append(component_number)
        else:
            trigger_type_0_serialnos.append(component_number)

    logger.info(f"Определено {len(trigger_type_1_serialnos)} записей с trigger_type=1.")
    logger.info(f"Определено {len(trigger_type_0_serialnos)} записей с trigger_type=0.")

    return trigger_type_1_serialnos, trigger_type_0_serialnos


def batch_update_trigger_type(client, database_name, serialnos, trigger_type, batch_size=1000):
    """
    Обновляет поле trigger_type в таблице OlapCube_VNV для списка serialno.
    Выполняет обновления пакетами размером batch_size.
    """
    try:
        total = len(serialnos)
        for i in range(0, total, batch_size):
            batch_serialnos = serialnos[i:i + batch_size]
            # Формируем строку для IN клаузулы
            serialnos_str = ",".join([f"'{sn}'" for sn in batch_serialnos])
            update_query = f"""
            ALTER TABLE {database_name}.OlapCube_VNV 
            UPDATE trigger_type = {trigger_type}
            WHERE serialno IN ({serialnos_str})
            """
            client.execute(update_query)
            logger.info(f"Обновлено trigger_type={trigger_type} для {len(batch_serialnos)} записей (батч {i//batch_size + 1}).")
    except Exception as e:
        logger.error(f"Ошибка при обновлении trigger_type={trigger_type}: {e}", exc_info=True)
        raise


def add_and_fill_trigger_type():
    """
    Добавляет поле trigger_type в таблицу OlapCube_VNV и заполняет его значениями на основе данных из таблицы Agregat.
    """
    try:
        # Шаг 1: Добавить поле trigger_type
        add_trigger_type_column(client, database_name)

        # Шаг 2: Извлечь данные из таблицы Agregat
        agregat_data = get_agregat_data(client, database_name)

        # Шаг 3: Определить trigger_type для каждого serialno
        trigger_type_1_serialnos, trigger_type_0_serialnos = determine_trigger_type(agregat_data)

        # Шаг 4: Обновить trigger_type=1
        if trigger_type_1_serialnos:
            batch_update_trigger_type(client, database_name, trigger_type_1_serialnos, trigger_type=1)
        else:
            logger.info("Нет записей для обновления trigger_type=1.")

        # Шаг 5: Обновить trigger_type=0
        if trigger_type_0_serialnos:
            batch_update_trigger_type(client, database_name, trigger_type_0_serialnos, trigger_type=0)
        else:
            logger.info("Нет записей для обновления trigger_type=0.")

        logger.info("Поле trigger_type успешно добавлено и заполнено в таблице OlapCube_VNV.")

    except Exception as e:
        logger.error(f"Ошибка при добавлении или обновлении поля trigger_type: {e}", exc_info=True)


def main():
    add_and_fill_trigger_type()


if __name__ == "__main__":
    main()


2025-07-08 17:03:19,691 - INFO - Поле trigger_type успешно добавлено в таблицу OlapCube_VNV.
2025-07-08 17:03:19,699 - INFO - Извлечено 1 записей из таблицы Agregat.
2025-07-08 17:03:19,699 - INFO - Определено 0 записей с trigger_type=1.
2025-07-08 17:03:19,700 - INFO - Определено 1 записей с trigger_type=0.
2025-07-08 17:03:19,700 - INFO - Нет записей для обновления trigger_type=1.
2025-07-08 17:03:19,705 - INFO - Обновлено trigger_type=0 для 1 записей (батч 1).
2025-07-08 17:03:19,706 - INFO - Поле trigger_type успешно добавлено и заполнено в таблице OlapCube_VNV.
